# **Datasets Merging and Normalization**

In [3]:
%load_ext autoreload
%autoreload 2

from utils import *

datasets_folder = './datasets'
verbosity=2

## **Input Data**

We load the datasets with the techniques used in the corresponding notebooks.

- Air pollution

In [4]:
pollution_data = read_and_preprocess_dataset(datasets_folder, 'pollution', v=verbosity)

Stations found: GIARDINI MARGHERITA, PORTA SAN FELICE, VIA CHIARINI
Splitting station "GIARDINI MARGHERITA"...
Splitting station "PORTA SAN FELICE"...
Splitting station "VIA CHIARINI"...


In [5]:
display(pollution_data['GIARDINI MARGHERITA']['NO2'].iloc[:2])
display(pollution_data['GIARDINI MARGHERITA']['NO2'].iloc[-2:])

,Agent_value
Date,
2019-01-01 00:00:00,29.0
2019-01-01 02:00:00,23.0


,Agent_value
Date,
2024-12-31 22:00:00,22.0
2024-12-31 23:00:00,21.0


- Traffic

In [ ]:
traffic_data = read_and_preprocess_dataset(datasets_folder, 'traffic', v=verbosity, radius=2)

Merging measurements files...
Merged 6 CSV files
Merging accuracies files...
Merged 6 CSV files
Location GIARDINI MARGHERITA: 44.482671138769533,11.35406170088398
 > Filtering close traffic data...
 > Summing up hour data...
Location PORTA SAN FELICE: 44.499059983334519,11.327526717440112
 > Filtering close traffic data...
 > Summing up hour data...
Location VIA CHIARINI: 44.499134335170289,11.285089594971216
 > Filtering close traffic data...
 > Summing up hour data...


In [7]:
display(traffic_data['GIARDINI MARGHERITA'].iloc[:2])
display(traffic_data['GIARDINI MARGHERITA'].iloc[-2:])

,Traffic_value
Date,
2019-01-01 00:00:00,10501.0
2019-01-01 01:00:00,16863.0


,Traffic_value
Date,
2024-12-31 22:00:00,4162.0
2024-12-31 23:00:00,3765.0


- Weather

In [8]:
weather_data = read_and_preprocess_dataset(datasets_folder, 'weather', v=verbosity)

Merging weather files...
Merged 6 CSV files


In [9]:
display(weather_data.iloc[:2])
display(weather_data.iloc[-2:])

,TAVG,PREC,RHAVG,RAD,W_SCAL_INT,W_VEC_DIR,LEAFW
Date,,,,,,,
2019-01-01 00:00:00,1.0,0.0,92.3,0.0,0.5,208.7,0.0
2019-01-01 01:00:00,0.3,0.0,93.6,0.0,0.5,280.0,0.0


,TAVG,PREC,RHAVG,RAD,W_SCAL_INT,W_VEC_DIR,LEAFW
Date,,,,,,,
2024-12-30 23:00:00,5.1,0.0,76.1,0.0,2.8,256.7,0.0
2024-12-31 00:00:00,5.1,0.0,75.0,0.0,2.8,258.3,0.0


**NOTE:** In each dataframe, the Date index represents the starting hour of each measurement. We used it because it was easier to work with.

## **Merge the datasets**

From the preprocessing, we saw that each station has its own data, mainly because of the traffic being different across them, but also because the agent values differ.\
We will then need to create a dataset for each agent-station pair. Some data will be duplicated.

We will merge the datasets for traffic, weather and pollution (target values) on the `Date` index.\
If the data to merge is hourly we can simply join the datasets using the index...

**NOTE:** only the process of one station is showed.

In [ ]:
merged_giardini_margherita = {}
merged_giardini_margherita['NO2'] = join_datasets(
    pollution_data['GIARDINI MARGHERITA']['NO2'],
    traffic_data['GIARDINI MARGHERITA'],
    weather_data,
    dropna=True
)

In [11]:
merged_giardini_margherita['NO2'].head(3)

,Agent_value,Traffic_value,TAVG,PREC,RHAVG,RAD,W_SCAL_INT,W_VEC_DIR,LEAFW
Date,,,,,,,,,
2019-01-01 00:00:00,29.0,10501.0,1.0,0.0,92.3,0.0,0.5,208.7,0.0
2019-01-01 02:00:00,23.0,15248.0,0.7,0.0,91.7,0.0,1.1,158.1,0.0
2019-01-01 03:00:00,29.0,9844.0,0.4,0.0,91.5,0.0,0.7,189.4,0.0


...but if we are considering a daily agent like *PM10*, as introduced in the [traffic notebook](./2-traffic_preprocessing.ipynb) and [weather notebook](./3-weather_preprocessing.ipynb), we should first convert the traffic and weather datasets to represent the daily informations by aggregating the hourly values. Then, we can merge those data together with the target values.

In [12]:
transform_traffic_to_daily_df # create bins for the traffic hours

<function utils.preprocessing.transform_traffic_to_daily_df(df: pandas.core.frame.DataFrame, bin_size=0, offset=0, dropna=True, **kwargs)>

In [13]:
transform_weather_to_daily_df # use appropriate aggregation functions for the daily features

<function utils.preprocessing.transform_weather_to_daily_df(df: pandas.core.frame.DataFrame, aggregations={}, max_min_columns=[], keep_splitted_columns=True, **kwargs)>

## **Scaling the features**

The features have different distributions, so we apply the transformation individually for each column. Both scaling to [0, 1], standardization or other functions can be used.

We also save the agents means and variances, to be able to go back to the original values.

In [14]:
normalized_giardini_margherita={}
# Applies `MinMaxScaler` to the specified columns, skipping `skip` columns.    
# If no skip columns are specified, all the columns are attempted to be scaled.
# If columns appear in `return_dist`, a dict with their mean and std will be returned.
normalized_giardini_margherita['NO2'], dist = normalize_columns(merged_giardini_margherita['NO2'], return_dists=['Agent_value'], scaler=StandardScaler())

In [15]:
print('Agent value distributions: ', dist['Agent_value'])
normalized_giardini_margherita['NO2'].head(3)

Agent value distributions:  {'mean': 17.14072246079708, 'std': 12.589960677710152}


,Agent_value,Traffic_value,TAVG,PREC,RHAVG,RAD,W_SCAL_INT,W_VEC_DIR,LEAFW
Date,,,,,,,,,
2019-01-01 00:00:00,0.941972,-0.429026,-1.741683,-0.13755,1.320229,-0.663642,-1.416150,0.046922,-0.464033
2019-01-01 02:00:00,0.465397,-0.000122,-1.776748,-0.13755,1.290525,-0.663642,-1.085620,-0.520981,-0.464033
2019-01-01 03:00:00,0.941972,-0.488387,-1.811813,-0.13755,1.280624,-0.663642,-1.305973,-0.169689,-0.464033


## **Encode date and time informations**

We also want to encode date and hour informations, to help the models learn that traffic is really small during the night or the weekends, and so on.

We can:
- one hot encode the hour/day/month: does not account for day 31 being close to day 1
- radial basis function: creates a feature for each possible value of the column to encode. For example, it would introduce 12 features if encoding the months (more accurate)
- sine/cosine: 1 sin feature and 1 cos feature for each column to encode

The year is not relevant.

We start by encoding the months using radial basis and the day using sine/cosine, introducing 12 features + 2 features. Using radial basis for the days would need 31 new columns and may not be needed.

*Source: [Three Approaches to Encoding Time Information as Features for ML Models](https://developer.nvidia.com/blog/three-approaches-to-encoding-time-information-as-features-for-ml-models/)*

In [16]:
method = 'radial_months-sin-cos_days_hours'

encoded_giardini_margherita = {}
# Using the given method, introduces new columns to encode the Date index.
encoded_giardini_margherita['NO2'] = encode_date_index(normalized_giardini_margherita['NO2'], method=method)

## **Final Dataset**

We end up with a dataset for each agent of each station, with input data and target values.

*Agent_value* has to be considered as $y_{true}$

In [17]:
encoded_giardini_margherita['NO2'].head(5)

,Agent_value,Traffic_value,TAVG,PREC,RHAVG,RAD,W_SCAL_INT,W_VEC_DIR,LEAFW,hour_sin,...,month_rbf_3,month_rbf_4,month_rbf_5,month_rbf_6,month_rbf_7,month_rbf_8,month_rbf_9,month_rbf_10,month_rbf_11,month_rbf_12
Date,,,,,,,,,,,,,,,,,,,,,
2019-01-01 00:00:00,0.941972,-0.429026,-1.741683,-0.13755,1.320229,-0.663642,-1.416150,0.046922,-0.464033,0.000000,...,0.000335,1.522998e-08,1.266417e-14,1.928750e-22,5.380186e-32,1.928750e-22,1.266417e-14,1.522998e-08,0.000335,0.135335
2019-01-01 02:00:00,0.465397,-0.000122,-1.776748,-0.13755,1.290525,-0.663642,-1.085620,-0.520981,-0.464033,0.500000,...,0.000335,1.522998e-08,1.266417e-14,1.928750e-22,5.380186e-32,1.928750e-22,1.266417e-14,1.522998e-08,0.000335,0.135335
2019-01-01 03:00:00,0.941972,-0.488387,-1.811813,-0.13755,1.280624,-0.663642,-1.305973,-0.169689,-0.464033,0.707107,...,0.000335,1.522998e-08,1.266417e-14,1.928750e-22,5.380186e-32,1.928750e-22,1.266417e-14,1.522998e-08,0.000335,0.135335
2019-01-01 04:00:00,0.703685,-0.817451,-1.741683,-0.13755,1.023191,-0.663642,-1.305973,-0.354875,-0.464033,0.866025,...,0.000335,1.522998e-08,1.266417e-14,1.928750e-22,5.380186e-32,1.928750e-22,1.266417e-14,1.522998e-08,0.000335,0.135335
2019-01-01 05:00:00,0.544826,-0.947017,-1.589735,-0.13755,0.696448,-0.663642,-0.700002,-0.600667,-0.464033,0.965926,...,0.000335,1.522998e-08,1.266417e-14,1.928750e-22,5.380186e-32,1.928750e-22,1.266417e-14,1.522998e-08,0.000335,0.135335
